# Forte Tutorial 3: Forte's determinant class

---

In this tutorial we are going to explore how to interact with forte in Jupyter notebooks using the Python API.

## Import modules
The first step necessary to interact with forte is to import psi4 and forte

In [3]:
import psi4
import forte
import forte.utils

First we will run psi4 using the function `forte.utils.psi4_scf`

In [4]:
# setup xyz geometry
geom = """
O
H 1 1.0
H 1 1.0 2 180.0
"""
(E_scf, wfn) = forte.utils.psi4_scf(geom,basis='sto-3g',reference='rhf')
print(f'SCF Energy = {E_scf}')

SCF Energy = -74.84159216002926


## Starting Forte and reading options

In [6]:
forte.startup()

from forte import forte_options

options = psi4.core.get_options() # options = psi4 option object
options.set_current_module('FORTE') # read options labeled 'FORTE'
forte_options.update_psi_options(options)

## Setting the molecular orbital spaces

In [9]:
# Setup forte and prepare the active space integral class
mos_spaces = {'FROZEN_DOCC' :     [1,0,0,0,0,0,0,0], # freeze the O 1s orbital
              'RESTRICTED_DOCC' : [1,0,0,0,0,1,0,0]}
mo_space_info = forte.make_mo_space_info_from_map(wfn,mos_spaces,[])

In [10]:
mo_space_info.size('ACTIVE')

4

## Building a `ForteIntegral` object to read integrals from psi4

In Forte there are two classes responsible for handling integrals:
- `ForteIntegral`: reads the integrals from psi4 and stores them in varios formats (conventional, density fitting, Cholesky, ...).
- `ActiveSpaceIntegrals`: stores a copy of all integrals and it is used by active space methods. This class only stores a subset of the integrals and includes an effective potential due to non-active doubly occupied orbitals.

We will first build the `ForteIntegral` object via the function `make_forte_integrals`

In [11]:
ints = forte.make_forte_integrals(wfn, options, mo_space_info)
print(f'Number of molecular orbitals: {ints.nmo()}')
print(f'Number of correlated molecular orbitals: {ints.ncmo()}')

# the space that defines the active orbitals. We select only the 'ACTIVE' part
active_space = 'ACTIVE'
# the space(s) with non-active doubly occupied orbitals
core_spaces = ['RESTRICTED_DOCC']

as_ints = forte.make_active_space_ints(mo_space_info, ints, active_space, core_spaces)

print(f'Frozen-core energy = {as_ints.frozen_core_energy()}')
print(f'Nuclear repulsion energy = {as_ints.nuclear_repulsion_energy()}')
print(f'Scalar energy = {as_ints.scalar_energy()}')

Number of molecular orbitals: 7
Number of correlated molecular orbitals: 6
Frozen-core energy = -60.56668027961306
Nuclear repulsion energy = 8.731423976054998
Scalar energy = -17.040281418154336


## Creating determinants

In [12]:
d = forte.Determinant()

In [22]:
nact = mo_space_info.size('ACTIVE')
d.str(nact)

d.create_alfa_bit(1)

1.0

In [24]:
d.str(nact)
d.create_alfa_bit(2)

-1.0

In [25]:
d.str(nact)

'|0++0>'

In [28]:
d.destroy_alfa_bit(2)

-1.0

In [29]:
d.str(nact)

'|0+00>'

## Creating the HF determinant

In [64]:
nirrep = mo_space_info.nirrep()
nactpi = mo_space_info.get_dimension('ACTIVE').to_tuple()

# compute the number of alpha electrons per irrep
nact_aelpi = wfn.nalphapi() - mo_space_info.get_dimension('FROZEN_DOCC') - mo_space_info.get_dimension('RESTRICTED_DOCC')
nact_aelpi = nact_aelpi.to_tuple()      
# compute the number of beta electrons per irrep
nact_belpi = wfn.nbetapi() - mo_space_info.get_dimension('FROZEN_DOCC') - mo_space_info.get_dimension('RESTRICTED_DOCC')
nact_belpi = nact_belpi.to_tuple()           

print(f'Number of alpha electrons per irrep: {nact_aelpi}')
print(f'Number of beta electrons per irrep:  {nact_belpi}')
print(f'Number of active orbtials per irrep: {nactpi}')

ref = forte.Determinant()

# we loop over each irrep and fill the occupied orbitals 
irrep_start = [sum(nactpi[:h]) for h in range(nirrep)]
for h in range(nirrep):
    for i in range(nact_aelpi[h]): ref.set_alfa_bit(irrep_start[h] + i, True)
    for i in range(nact_belpi[h]): ref.set_beta_bit(irrep_start[h] + i, True)        
    
print(f'Reference determinant: {ref.str(nact)}')

Number of alpha electrons per irrep: (0, 0, 0, 0, 0, 0, 1, 1)
Number of beta electrons per irrep:  (0, 0, 0, 0, 0, 0, 1, 1)
Number of active orbtials per irrep: (1, 0, 0, 0, 0, 1, 1, 1)
Reference determinant: |0022>


We can now compute the energy of the determinant as $\langle \Phi | \hat{H} | \Phi \rangle$ using the `slater_rules` function in the `ActiveSpaceIntegrals` class

In [58]:
as_ints.slater_rules(ref,ref) + as_ints.scalar_energy() + as_ints.nuclear_repulsion_energy() 

-74.84159216002939

We can also print all the integrals at once (see the `output.dat` file)

In [97]:
import itertools
import functools

dets = []
orbs = range(nact)

# get the symmetry of each active orbital
act_sym = mo_space_info.symmetry('ACTIVE')

nact_ael = sum(nact_aelpi)
nact_bel = sum(nact_belpi)
print(f'Number of alpha electrons: {nact_ael}')
print(f'Number of beta electrons:  {nact_bel}')

# specify the target symmetry
sym = 0

# generate all the alpha strings
for astr in itertools.combinations(orbs, nact_ael):
    # compute the symmetry of the alpha string
    asym = functools.reduce(lambda i, j:  act_sym[i] ^ act_sym[j], astr)
    # generate all the beta strings
    for bstr in itertools.combinations(orbs, nact_bel):
        # compute the symmetry of the beta string
        bsym = functools.reduce(lambda i, j:  act_sym[i] ^ act_sym[j], bstr)    
        # if the determinant has the correct symmetry save it
        if (asym ^ bsym) == sym:
            d = forte.Determinant()
            for i in astr: d.set_alfa_bit(i, True)
            for i in bstr: d.set_beta_bit(i, True)                
            dets.append(d)

print(f'==> List of FCI determinants <==')
for d in dets:
    print(f'{d.str(64)}')

Number of alpha electrons: 2
Number of beta electrons:  2
==> List of FCI determinants <==
|2200000000000000000000000000000000000000000000000000000000000000>
|2020000000000000000000000000000000000000000000000000000000000000>
|2002000000000000000000000000000000000000000000000000000000000000>
|0220000000000000000000000000000000000000000000000000000000000000>
|0202000000000000000000000000000000000000000000000000000000000000>
|0022000000000000000000000000000000000000000000000000000000000000>


In [123]:
import numpy as np

ndets = len(dets)
H = np.ndarray((ndets,ndets))
for I, detI in enumerate(dets):
    H[I][I] = as_ints.slater_rules(detI,detI) # diagonal term
    for J, detJ in enumerate(dets[:I]):
        HIJ = as_ints.slater_rules(detI,detJ) # off-diagonal term (only upper half)
        H[I][J] = H[J][I] = HIJ

# for I, detI in enumerate(dets):
#     for J, detJ in enumerate(dets):
#         H[I][J] = as_ints.slater_rules(detI,detJ)
   
print(H)
evals, evecs = np.linalg.eigh(H)


psi4_fci = -74.846380133240530
print(f'FCI Energy = {evals[0] + as_ints.scalar_energy() + as_ints.nuclear_repulsion_energy()}')
print(f'FCI Energy Error = {evals[0] + as_ints.scalar_energy() + as_ints.nuclear_repulsion_energy()- psi4_fci}')

index_hf = dets.index(ref)
print(f'Index of the HF determinant in the FCI vector {index_hf}')

[[-6.39776098e+01  2.76136365e-02  2.76136365e-02  4.26858693e-02
   4.26858693e-02  0.00000000e+00]
 [ 2.76136365e-02 -6.57203915e+01  4.74444451e-02  1.55359165e-01
   0.00000000e+00  4.26858693e-02]
 [ 2.76136365e-02  4.74444451e-02 -6.57203915e+01  0.00000000e+00
   1.55359165e-01  4.26858693e-02]
 [ 4.26858693e-02  1.55359165e-01  0.00000000e+00 -6.51549711e+01
   4.74444451e-02  2.76136365e-02]
 [ 4.26858693e-02  0.00000000e+00  1.55359165e-01  4.74444451e-02
  -6.51549711e+01  2.76136365e-02]
 [ 0.00000000e+00  4.26858693e-02  4.26858693e-02  2.76136365e-02
   2.76136365e-02 -6.65327347e+01]]
FCI Energy = -74.84638013325272
FCI Energy Error = -1.2192913345643319e-11
Index of the HF determinant in the FCI vector 5


In [ ]:
forte.cleanup()